In [1]:
hooktheory_midi_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/ProcessedData/midi_from_json_songs"
# model_output_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Evaluation/data_for_testing/model_output/topk15-t1.0-ngram0"
# model_output_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/Ideas/data/output/topk15-t1.0-ngram0"

model_output_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Evaluation/data_for_testing/model_output/data for visualization/lora gpt2"

In [2]:
import pretty_midi
import numpy as np
from collections import Counter
import os
import plotly.express as px

In [3]:
# current_idx = 0

# indices_to_analyze = [
#     0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
#     # 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
#     # 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
#     # 100, 101, 102, 103, 104, 105, 106, 107, 108, 109
# ]

In [4]:
def load_midi(file_path):
    return pretty_midi.PrettyMIDI(file_path)

def get_notes(midi_data):
    notes = []
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                notes.append(note)
    return sorted(notes, key=lambda note: note.start)

def pitch_counts(notes):
    pitches = [note.pitch for note in notes]
    return dict(Counter(pitches))

def pitch_class_distribution(notes):
    pitch_classes = [note.pitch % 12 for note in notes]
    return dict(Counter(pitch_classes))

def pitch_class_transition_matrix(notes):
    pitch_classes = [note.pitch % 12 for note in notes]
    transition_matrix = np.zeros((12, 12))
    for i in range(len(pitch_classes) - 1):
        transition_matrix[pitch_classes[i], pitch_classes[i + 1]] += 1
    return transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

def pitch_range(notes):
    pitches = [note.pitch for note in notes]
    return max(pitches) - min(pitches)

def average_pitch_intervals(notes):
    pitch_intervals = [notes[i + 1].pitch - notes[i].pitch for i in range(len(notes) - 1)]
    return np.mean(pitch_intervals), np.std(pitch_intervals)

def average_inter_onset_intervals(notes):
    inter_onset_intervals = [notes[i + 1].start - notes[i].start for i in range(len(notes) - 1)]
    return np.mean(inter_onset_intervals), np.std(inter_onset_intervals)

def note_count(notes):
    return len(notes)

def note_length_transition_matrix(notes):
    note_lengths = [note.end - note.start for note in notes]
    unique_lengths = sorted(set(note_lengths))
    length_to_idx = {length: idx for idx, length in enumerate(unique_lengths)}
    transition_matrix = np.zeros((len(unique_lengths), len(unique_lengths)))
    
    for i in range(len(note_lengths) - 1):
        current_length = note_lengths[i]
        next_length = note_lengths[i + 1]
        transition_matrix[length_to_idx[current_length], length_to_idx[next_length]] += 1
    
    return transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

def analyze_midi(file_path):
    midi_data = load_midi(file_path)
    notes = get_notes(midi_data)

    analysis = {}
    
    if len(notes) != 0:
        analysis = {
            'pitch_counts': pitch_counts(notes),
            'pitch_class_distribution': pitch_class_distribution(notes),
            # 'pitch_class_transition_matrix': pitch_class_transition_matrix(notes),
            # 'pitch_range': pitch_range(notes),
            # 'average_pitch_intervals': average_pitch_intervals(notes),
            # 'average_inter_onset_intervals': average_inter_onset_intervals(notes),
            'note_count': note_count(notes),
            # 'note_length_transition_matrix': note_length_transition_matrix(notes)
        }
    else:
        pass
    return analysis

def analyze_midi_folder(folder_path, max_file_count=10000):
    results = {}

    current_file_count = 0
    # Walk recursively
    # for file_name in os.listdir(folder_path):
    #     if file_name.endswith('.mid') or file_name.endswith('.midi'):
    #         current_file_count += 1
    #         if current_file_count > max_file_count:
    #             break
    #         else:
    #             file_path = os.path.join(folder_path, file_name)
    #             print(f"Analyzing file {current_file_count} of {max_file_count} - {file_name}")
    #             results[file_name] = analyze_midi(file_path)
    global current_idx
    global indices_to_analyze

    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            # if current_idx not in indices_to_analyze:
            #     current_idx += 1
            #     continue
            # else:
            #     current_idx += 1
                
            if file_name.endswith('.mid') or file_name.endswith('.midi'):
                current_file_count += 1
                if current_file_count > max_file_count:
                    break
                else:
                    file_path = os.path.join(root, file_name)
                    print(f"Analyzing file {current_file_count} of {max_file_count} - {file_name}")
                    results[file_name] = analyze_midi(file_path)
    
    current_idx = 0
    
    return results

In [5]:
def average_analysis_results(analysis_results):
    average_results = {
        "pitch_counts": dict[int, float](),
        "pitch_class_distribution": dict[int, float](),
        "note_count": int(0)
    }
    
    n_files = len(analysis_results)

    for file_name, analysis in analysis_results.items():
        for key, value in analysis.items():
            if key == 'pitch_counts':
                for pitch, v in value.items():
                    if pitch not in average_results["pitch_counts"]:
                        average_results["pitch_counts"][pitch] = v / n_files
                    else:
                        average_results["pitch_counts"][pitch] += v / n_files
            elif key == 'pitch_class_distribution':
                for pitch_class, v in value.items():
                    if pitch_class not in average_results["pitch_class_distribution"]:
                        average_results["pitch_class_distribution"][pitch_class] = v / n_files
                    else:
                        average_results["pitch_class_distribution"][pitch_class] += v / n_files
            elif key == 'note_count':
                average_results[key] += value / n_files
            else:
                pass
    
    return average_results

# Hooktheory

In [6]:
folder_path = hooktheory_midi_dir
n_file_to_analyze = 29038
n_file_to_analyze = 100

analysis_results = analyze_midi_folder(folder_path, n_file_to_analyze)

Analyzing file 1 of 100 - d_deerhunter_back-to-the-middle_Solo_AQodPXyKoDl.mid
Analyzing file 2 of 100 - t_the-beatles_a-hard-days-night_Intro_ZwxK_QNbxed.mid
Analyzing file 3 of 100 - n_niall-horan_still_Bridge_jDgXdDVegKl.mid
Analyzing file 4 of 100 - a_ashe_love-is-not-enough_Pre-Chorus_yvgPQdedxYq.mid
Analyzing file 5 of 100 - b_bryan-scary_operaland_Chorus_DpgvRAkdgad.mid
Analyzing file 6 of 100 - j_jimmy-fontanez_urban-lullaby_Verse_Wegl_wa-mrY.mid
Analyzing file 7 of 100 - h_hirohiko-takayama_hudsons-adventure-island-iii---thunder-clash_Instrumental_yvmrLZGzxOW.mid
Analyzing file 8 of 100 - h_hitomi-yaida_ashita-kara-no-tegami_Intro and Verse_AaoGbakPxeQ.mid
Analyzing file 9 of 100 - b_billy-idol_white-wedding_Chorus_eWxLdzOpxaK.mid
Analyzing file 10 of 100 - l_lawrence_the-heartburn-song_Verse_ROmNkROngNw.mid
Analyzing file 11 of 100 - m_miguel_goingtohell_Verse_zngREQNMmJj.mid
Analyzing file 12 of 100 - b_bensound_elevator-bossa-nova_Intro and Verse_d_gwnZ_QmGV.mid
Analyzing f

In [7]:
n_file_to_print = 1

current_file = 0

# Print results
for file_name, analysis in analysis_results.items():
    if current_file < n_file_to_print:
        current_file += 1
        print(f"{current_file} of {n_file_to_analyze} result - {file_name}:")

        for key, value in analysis.items():
            print(f"  {key}: {value}")
    else:
        break

1 of 100 result - d_deerhunter_back-to-the-middle_Solo_AQodPXyKoDl.mid:
  pitch_counts: {71: 14, 59: 2, 62: 2, 66: 16, 35: 2, 73: 6, 68: 24, 57: 2, 61: 12, 64: 14, 33: 2, 69: 2, 42: 2, 40: 2, 37: 2}
  pitch_class_distribution: {11: 18, 2: 2, 6: 18, 1: 20, 8: 24, 9: 6, 4: 16}
  note_count: 104


In [8]:
average_results = average_analysis_results(analysis_results)

# Print average results
print("Average results:")
for key, value in average_results.items():
    print(f"  {key}: {value}")

# Save average results to file
# with open('average_results.json', 'w') as f:
#     json.dump(average_results, f, indent=4)

# Save analysis results to file
# with open('analysis_results.json', 'w') as f:
#     json.dump(analysis_results, f, indent=4)

Average results:
  pitch_counts: {71: 4.09, 59: 4.319999999999999, 62: 7.209999999999999, 66: 5.18, 35: 1.4400000000000004, 73: 2.08, 68: 4.749999999999999, 57: 3.4499999999999984, 61: 4.339999999999998, 64: 5.969999999999999, 33: 1.3500000000000005, 69: 5.6099999999999985, 42: 0.39999999999999997, 40: 0.6300000000000001, 37: 0.9000000000000001, 65: 6.089999999999997, 67: 7.349999999999999, 72: 3.4899999999999993, 38: 1.0800000000000005, 60: 4.889999999999995, 77: 1.1900000000000004, 75: 1.5600000000000005, 44: 0.21000000000000002, 56: 1.6000000000000003, 63: 4.509999999999997, 32: 0.6800000000000002, 58: 3.549999999999999, 51: 0.6700000000000002, 55: 2.6399999999999992, 27: 0.48, 34: 1.3600000000000005, 36: 1.0500000000000003, 76: 1.4200000000000006, 74: 2.609999999999999, 78: 1.0900000000000003, 79: 0.9400000000000003, 81: 0.37, 82: 0.3800000000000001, 70: 3.3299999999999987, 45: 0.25, 80: 0.4700000000000001, 83: 0.19999999999999998, 85: 0.21000000000000002, 87: 0.07, 39: 0.480000000

In [9]:
# Horizontal bar chart for pitch counts using plotly express (with y is the pitch and x is the count)
fig = px.bar(
    x=list(average_results['pitch_counts'].values()), 
    y=list(average_results['pitch_counts'].keys()), 
    orientation='h'
)

fig.update_layout(
    title=f'Hooktheory (n = {n_file_to_analyze})', 
    xaxis_title='Số lần xuất hiện trung bình',
    yaxis_title='Cao độ'
)

# Show labels 0, 5, 10, 15, ... in y-axis
y_axis_tickvals = list(range(0, max(average_results['pitch_counts'].keys()) + 1, 5))
fig.update_yaxes(tickvals=y_axis_tickvals)

# Show labels 0, 1, 2, 3, ... in x-axis
x_axis_tickvals = list(range(
    0, 
    int(max(average_results['pitch_counts'].values())) + 1 + 1
))
fig.update_xaxes(tickvals=x_axis_tickvals)

# Change the color set to make it easier to see when printed in black and white
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)

# Change the background color to make it easier to see when printed in black and white
fig.update_layout(plot_bgcolor='white')

# Show the vertical grid lines
fig.update_layout(xaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgb(158,202,225)'))

# Make the colums overlay the grid lines
fig.update_layout(barmode='overlay')

# Make the font size bigger
fig.update_layout(font=dict(size=13))

# Make the plot ratio x:y
fig.update_layout(
    autosize=False,
    width=450,
    height=1200,
)

fig.show()


# Pitch class distribution
fig = px.bar(x=list(average_results['pitch_class_distribution'].keys()), y=list(average_results['pitch_class_distribution'].values()))
fig.update_layout(
    title=f'Hooktheory (n = {n_file_to_analyze})', 
    yaxis_title='Số lần xuất hiện trung bình',
    xaxis_title='Cao độ cơ bản'
)

# Show labels 0, 1, 2, 3, ... in x-axis
x_axis_tickvals = list(range(0, max(average_results['pitch_class_distribution'].keys()) + 1))
fig.update_xaxes(tickvals=x_axis_tickvals)

# Change the color set to make it easier to see when printed in black and white
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)

# Change the background color to make it easier to see when printed in black and white
fig.update_layout(plot_bgcolor='white')

# Show the horizontal grid lines
fig.update_layout(yaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgb(158,202,225)'))

# Make the font size bigger
fig.update_layout(font=dict(size=13))

# Make the plot ratio x:y
fig.update_layout(
    autosize=False,
    width=650,
    height=400,
)

fig.show()

# Note count
print(f"Số lượng nốt nhạc trung bình trong một đoạn nhạc từ Hooktheory (n = {n_file_to_analyze}): {average_results['note_count']}")

Số lượng nốt nhạc trung bình trong một đoạn nhạc từ Hooktheory (n = 100): 116.44000000000007


# Model Output

In [10]:
folder_path = model_output_dir

# Count midi files in the folder path
n_file_to_analyze = 0
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.endswith('.mid') or file_name.endswith('.midi'):
            n_file_to_analyze += 1

analysis_results = analyze_midi_folder(folder_path, n_file_to_analyze)

Analyzing file 1 of 302 - 166_2024_07_28_11_pm_164.mid
Analyzing file 2 of 302 - 257_2024_07_29_10_am_82.mid
Analyzing file 3 of 302 - 71_2024_07_28_09_am_69.mid
Analyzing file 4 of 302 - 258_2024_07_29_10_am_83.mid
Analyzing file 5 of 302 - 230_2024_07_29_06_am_55.mid
Analyzing file 6 of 302 - 107_2024_07_28_03_pm_105.mid
Analyzing file 7 of 302 - 52_2024_07_28_06_am_50.mid
Analyzing file 8 of 302 - 118_2024_07_28_04_pm_116.mid
Analyzing file 9 of 302 - 210_2024_07_29_04_am_35.mid
Analyzing file 10 of 302 - 97_2024_07_28_01_pm_95.mid
Analyzing file 11 of 302 - 249_2024_07_29_09_am_74.mid
Analyzing file 12 of 302 - 0_2024_07_27_11_pm_2.mid
Analyzing file 13 of 302 - 55_2024_07_28_07_am_53.mid
Analyzing file 14 of 302 - 244_2024_07_29_09_am_69.mid
Analyzing file 15 of 302 - 263_2024_07_29_11_am_88.mid
Analyzing file 16 of 302 - 189_2024_07_29_01_am_14.mid
Analyzing file 17 of 302 - 161_2024_07_28_10_pm_159.mid
Analyzing file 18 of 302 - 235_2024_07_29_07_am_60.mid
Analyzing file 19 of 3

In [11]:
n_file_to_print = 1

current_file = 0

# Print results
for file_name, analysis in analysis_results.items():
    if current_file < n_file_to_print:
        current_file += 1
        print(f"{current_file} of {n_file_to_analyze} result - {file_name}:")

        for key, value in analysis.items():
            print(f"  {key}: {value}")
    else:
        break

1 of 302 result - 166_2024_07_28_11_pm_164.mid:
  pitch_counts: {57: 15, 50: 20, 69: 10, 53: 7, 62: 11, 60: 21, 67: 24, 72: 17, 74: 14, 59: 9, 65: 14, 52: 16, 48: 11, 45: 6, 55: 5, 36: 1, 43: 2, 77: 4, 64: 49, 70: 2, 76: 46, 38: 1, 58: 1, 44: 1, 40: 1, 56: 1, 41: 2, 93: 1, 79: 1}
  pitch_class_distribution: {9: 32, 2: 46, 5: 27, 0: 50, 7: 32, 11: 9, 4: 112, 10: 3, 8: 2}
  note_count: 313


In [12]:
average_results = average_analysis_results(analysis_results)

# Print average results
print("Average results:")
for key, value in average_results.items():
    print(f"  {key}: {value}")

# Save average results to file
# with open('average_results.json', 'w') as f:
#     json.dump(average_results, f, indent=4)

# Save analysis results to file
# with open('analysis_results.json', 'w') as f:
#     json.dump(analysis_results, f, indent=4)

Average results:
  pitch_counts: {57: 12.97350993377484, 50: 12.264900662251664, 69: 21.96026490066221, 53: 7.953642384105951, 62: 16.268211920529776, 60: 18.062913907284738, 67: 16.486754966887403, 72: 12.754966887417206, 74: 12.89735099337747, 59: 10.887417218543028, 65: 8.476821192052974, 52: 14.185430463576154, 48: 11.23841059602648, 45: 5.556291390728469, 55: 14.04966887417217, 36: 5.619205298013237, 43: 5.4238410596026405, 77: 4.291390728476817, 64: 24.13907284768208, 70: 0.7516556291390732, 76: 19.347682119205288, 38: 2.139072847682117, 58: 0.5397350993377485, 44: 0.7052980132450336, 40: 2.596026490066221, 56: 1.0364238410596032, 41: 4.3774834437086065, 93: 0.029801324503311258, 79: 4.003311258278141, 75: 0.7218543046357621, 51: 1.778145695364237, 46: 1.4834437086092727, 63: 0.5827814569536426, 87: 0.17880794701986766, 66: 1.4437086092715234, 47: 2.4205298013245025, 71: 5.834437086092711, 54: 1.0827814569536425, 61: 0.7582781456953641, 49: 0.7980132450331131, 81: 1.7086092715231

In [13]:
# Horizontal bar chart for pitch counts using plotly express (with y is the pitch and x is the count)
fig = px.bar(
    x=list(average_results['pitch_counts'].values()), 
    y=list(average_results['pitch_counts'].keys()), 
    orientation='h'
)

fig.update_layout(
    title=f'GPT (n = {n_file_to_analyze})', 
    xaxis_title='Số lần xuất hiện trung bình',
    yaxis_title='Cao độ'
)

# Show labels 0, 5, 10, 15, ... in y-axis
y_axis_tickvals = list(range(0, max(average_results['pitch_counts'].keys()) + 1, 5))
fig.update_yaxes(tickvals=y_axis_tickvals)

# Show labels 0, 1, 2, 3, ... in x-axis
x_axis_tickvals = list(range(
    0, 
    int(max(average_results['pitch_counts'].values())) + 1 + 1
))
fig.update_xaxes(tickvals=x_axis_tickvals)

# Change the color set to make it easier to see when printed in black and white
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)

# Change the background color to make it easier to see when printed in black and white
fig.update_layout(plot_bgcolor='white')

# Show the vertical grid lines
fig.update_layout(xaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgb(158,202,225)'))

# Make the colums overlay the grid lines
fig.update_layout(barmode='overlay')

# Make the font size bigger
fig.update_layout(font=dict(size=13))

# Make the plot ratio x:y
fig.update_layout(
    autosize=False,
    width=450,
    height=1200,
)

fig.show()


# Pitch class distribution
fig = px.bar(x=list(average_results['pitch_class_distribution'].keys()), y=list(average_results['pitch_class_distribution'].values()))
fig.update_layout(
    title=f'GPT (n = {n_file_to_analyze})', 
    yaxis_title='Số lần xuất hiện trung bình',
    xaxis_title='Cao độ cơ bản'
)

# Show labels 0, 1, 2, 3, ... in x-axis
x_axis_tickvals = list(range(0, max(average_results['pitch_class_distribution'].keys()) + 1))
fig.update_xaxes(tickvals=x_axis_tickvals)

# Change the color set to make it easier to see when printed in black and white
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)

# Change the background color to make it easier to see when printed in black and white
fig.update_layout(plot_bgcolor='white')

# Show the horizontal grid lines
fig.update_layout(yaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgb(158,202,225)'))

# Make the font size bigger
fig.update_layout(font=dict(size=13))

# Make the plot ratio x:y
fig.update_layout(
    autosize=False,
    width=650,
    height=400,
)

fig.show()

# Note count
print(f"Số lượng nốt nhạc trung bình trong một đoạn nhạc từ GPT (n = {n_file_to_analyze}): {average_results['note_count']}")

Số lượng nốt nhạc trung bình trong một đoạn nhạc từ GPT (n = 302): 303.0728476821189


# Model Output (C major & A minor key only)

In [14]:
# Select the pitches to analyze (C major scale and A minor scale)
selected_pitchs = [
    0, # C
    2, # D
    4, # E
    5, # F
    7, # G
    9, # A
    11 # B
]

# AVARAGE PITCH COUNTS
# Sort average_results['pitch_counts'] by pitch
average_results['pitch_counts'] = dict(sorted(average_results['pitch_counts'].items()))

print(average_results['pitch_counts'])

# Filter the pitch that pitch mod any of the selected pitches equals 0
filtered_pitch_counts = {
    str(pitch): count 
    for pitch, count in average_results['pitch_counts'].items() 
    if pitch % 12 in selected_pitchs
}

print(filtered_pitch_counts)

# AVARAGE PITCH CLASS DISTRIBUTION
# Sort average_results['pitch_class_distribution'] by pitch class
average_results['pitch_class_distribution'] = dict(sorted(average_results['pitch_class_distribution'].items()))

print(average_results['pitch_class_distribution'])

# Filter the pitch class that is in the selected pitch classes
filtered_pitch_class_distribution = {
    str(pitch_class): count 
    for pitch_class, count in average_results['pitch_class_distribution'].items() 
    if pitch_class in selected_pitchs
}

print(filtered_pitch_class_distribution)

{24: 0.05629139072847682, 25: 0.11589403973509939, 26: 0.07284768211920532, 28: 0.029801324503311258, 29: 0.049668874172185434, 31: 0.13576158940397354, 32: 0.1423841059602649, 33: 2.4271523178807928, 34: 0.043046357615894044, 35: 1.4072847682119185, 36: 5.619205298013237, 37: 0.34768211920529785, 38: 2.139072847682117, 39: 0.21192052980132448, 40: 2.596026490066221, 41: 4.3774834437086065, 42: 1.8543046357615884, 43: 5.4238410596026405, 44: 0.7052980132450336, 45: 5.556291390728469, 46: 1.4834437086092727, 47: 2.4205298013245025, 48: 11.23841059602648, 49: 0.7980132450331131, 50: 12.264900662251664, 51: 1.778145695364237, 52: 14.185430463576154, 53: 7.953642384105951, 54: 1.0827814569536425, 55: 14.04966887417217, 56: 1.0364238410596032, 57: 12.97350993377484, 58: 0.5397350993377485, 59: 10.887417218543028, 60: 18.062913907284738, 61: 0.7582781456953641, 62: 16.268211920529776, 63: 0.5827814569536426, 64: 24.13907284768208, 65: 8.476821192052974, 66: 1.4437086092715234, 67: 16.4867549

In [20]:
# Horizontal bar chart for filtered pitch counts using plotly express (with y is the pitch and x is the count)
fig = px.bar(
    x=list(filtered_pitch_counts.values()), 
    y=list(filtered_pitch_counts.keys()), 
    orientation='h'
)

fig.update_layout(
    title=f'GPT (n = {n_file_to_analyze})', 
    xaxis_title='Số lần xuất hiện trung bình',
    yaxis_title='Cao độ'
)

# Show labels 0, 5, 10, 15, ... in y-axis in string format
# y_axis_tickvals = list(range(0, max(filtered_pitch_counts.keys()) + 1, 5))
y_axis_tickvals = list(map(str, [x for x in filtered_pitch_counts.keys()]))

fig.update_yaxes(tickvals=y_axis_tickvals)

# Show labels 0, 1, 2, 3, ... in x-axis
x_axis_tickvals = list(range(
    0, 
    int(max(filtered_pitch_counts.values())) + 1 + 1
))

fig.update_xaxes(tickvals=x_axis_tickvals)

# Change the color set to make it easier to see when printed in black and white
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)

# Change the background color to make it easier to see when printed in black and white
fig.update_layout(plot_bgcolor='white')

# Show the vertical grid lines
fig.update_layout(xaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgb(158,202,225)'))

# Make the columns overlay the grid lines
fig.update_layout(barmode='overlay')

# Make the font size bigger
fig.update_layout(font=dict(size=13))

# Make the plot ratio x:y
fig.update_layout(
    autosize=False,
    width=450,
    height=900,
)

fig.show()

# Filtered pitch class distribution
fig = px.bar(
    x=list(filtered_pitch_class_distribution.keys()),
    y=list(filtered_pitch_class_distribution.values())
)

fig.update_layout(
    title=f'GPT (n = {n_file_to_analyze})', 
    yaxis_title='Số lần xuất hiện trung bình',
    xaxis_title='Cao độ cơ bản'
)

# Show labels 0, 1, 2, 3, ... in x-axis
# x_axis_tickvals = list(range(0, max(filtered_pitch_class_distribution.keys()) + 1))

# fig.update_xaxes(tickvals=x_axis_tickvals)

# Change the color set to make it easier to see when printed in black and white
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)

# Change the background color to make it easier to see when printed in black and white
fig.update_layout(plot_bgcolor='white')

# Show the horizontal grid lines
fig.update_layout(yaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgb(158,202,225)'))

# Make the font size bigger
fig.update_layout(font=dict(size=13))

# Make the plot ratio x:y
fig.update_layout(
    autosize=False,
    width=650,
    height=400,
)

fig.show()